<a href="https://colab.research.google.com/github/NehaKumari500092077/Machine-Learning-Lab/blob/main/Session_4_Assignments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Lab Assignment 3**

### <font color='blue'>Objective: </font>

In this assignment, you will learn how to handle and preprocess a real-world dataset, including dealing with duplicates, missing values, and feature scaling. You'll implement the Naive Bayes classifier from scratch, selecting appropriate likelihood distributions for different feature types. You'll also compare your implementation with an existing model (MixedNB) and assess its performance using various metrics. Finally, you'll learn to effectively evaluate and visualize the results to communicate your findings.

### <font color='blue'> Task - 1 [Marks 0] </font>:
Load the [Secondary Mushroom](https://archive.ics.uci.edu/dataset/848/secondary+mushroom+dataset) dataset using Pandas. This dataset has 20 features, and the target variable is binary, which indicates if the mushroom is edible or not


### <font color='blue'> Task - 2 [Marks 1] </font>:
:
Check if there are duplicate entries in the data and missing values of features. Remove duplicate entries and handle entries having missing feature values using Imputation method(Like Mean, Mediaan, Mode etc).  

### <font color='blue'> Task - 3 [Marks 3] </font>:

1.Preprocess the dataset as required, i.e. feature scaling or standardization

2.Is the dataset balanced or imbalanced?

3.Split data into training and test set

### <font color='blue'> Task - 4 [Marks 7] </font>:

Implement Naive Bayes’ classifier from scratch on this dataset, by appropriately choosing the likelihood distribution for each feature. This dataset has mixed feature types (i.e. continuous and categorical features), and the likelihood distribution of each feature must consider the corresponding feature type. Mention the type of distribution you chose for each feature’s likelihood.  


### <font color='blue'> Task - 5 [Marks 2] </font>:

Report the classification performance using the appropriate metrics (accuracy, precision, recall, confusion matrix, AUPRC) using suitable plots.

### <font color='blue'> Task - 6 [Marks 5] </font>:
Fit a Naive Bayes’ model for this dataset using MixedNB (from the package https://pypi.org/project/mixed-naive-bayes/)



### <font color='blue'> Task - 7 [Marks 2] </font>:

Compare the performance obtained by your implementation with that obtained using MixedNB